# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = '../WeatherPy/output_data/clean_cities.csv'
cities_df = pd.read_csv(csv_path)
cities_df.head()

,City Name,Latitude,Longitude,Daily High,Humidity,Cloudiness,Wind Speed,Country,Date
0,Asău,46.43,26.40,61.00,87,0,4.16,RO,2020-07-11
1,Mataura,-46.19,168.86,44.15,93,69,3.31,NZ,2020-07-11
2,Salalah,17.02,54.09,80.60,94,90,3.36,OM,2020-07-11
3,Cape Town,-33.93,18.42,54.00,81,20,9.17,ZA,2020-07-11
4,Menongue,-14.66,17.69,55.45,37,0,4.43,AO,2020-07-11


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)
locations = cities_df[['Latitude','Longitude']].astype(float)


In [4]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(zoom_level=3,center=(30,100))

heat_layer = gmaps.heatmap_layer(locations, weights=cities_df.Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1.5)

fig.add_layer(heat_layer)



In [5]:
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
# Note: Feel free to adjust to your specifications but be sure to limit the number of rows 
# returned by your API requests to a reasonable number.

finest_cities = cities_df.loc[
    (cities_df['Daily High'] < 75) &
    (cities_df['Daily High'] > 65) &
    (cities_df['Wind Speed'] < 10) &
    (cities_df['Cloudiness'] < 2) &
    (cities_df['Humidity'] < 60)
].copy()

finest_cities

,City Name,Latitude,Longitude,Daily High,Humidity,Cloudiness,Wind Speed,Country,Date
79,Bataiporã,-22.30,-53.27,71.55,55,0,5.68,BR,2020-07-11
223,Zabul Province,32.25,67.25,67.71,23,0,5.28,AF,2020-07-11
239,Zhezkazgan,47.80,67.71,71.73,44,0,9.06,KZ,2020-07-11
255,Arinos,-15.92,-46.11,70.20,43,0,2.95,BR,2020-07-11
391,Corinth,37.94,22.96,73.99,46,0,1.99,GR,2020-07-11
510,Elena,42.93,25.88,71.60,56,0,5.82,BG,2020-07-11


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = 'Store'

finest_cities['Hotel Name'] = ''

locations = finest_cities[['Latitude','Longitude']]

lat = locations.iloc[0,0]
lng = locations.iloc[0,1]
loc = str(lat)+','+str(lng)
loc

'-22.3,-53.27'

In [8]:
finest_cities

,City Name,Latitude,Longitude,Daily High,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
79,Bataiporã,-22.30,-53.27,71.55,55,0,5.68,BR,2020-07-11,
223,Zabul Province,32.25,67.25,67.71,23,0,5.28,AF,2020-07-11,
239,Zhezkazgan,47.80,67.71,71.73,44,0,9.06,KZ,2020-07-11,
255,Arinos,-15.92,-46.11,70.20,43,0,2.95,BR,2020-07-11,
391,Corinth,37.94,22.96,73.99,46,0,1.99,GR,2020-07-11,
510,Elena,42.93,25.88,71.60,56,0,5.82,BG,2020-07-11,


In [9]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    #"location": "",  
    "rankby": "distance",
    "type": "hotel",
    "key": g_key,
}
params['location'] = loc
response = requests.get(base_url, params=params).json()
response['results'][0]['name']

# use iterrows to iterate through pandas dataframe
for index, row in finest_cities.iterrows():
    lat = finest_cities.loc[index, 'Latitude']
    lng = finest_cities.loc[index, 'Longitude']
    loc = str(lat)+','+str(lng)
    params['location'] = loc
    response = requests.get(base_url, params=params).json()
    try:
        name = response['results'][0]['name']
        print(f"Closest hotel to {finest_cities.loc[index, 'City Name']} is {name}.")
        finest_cities.loc[index, 'Hotel Name'] = name
    except (KeyError, IndexError):
        print("Missing hotel... skipping.")
        
    print("------------")


Closest hotel to Bataiporã is Câmara Municipal de Bataypora.
------------
Missing hotel... skipping.
------------
Closest hotel to Zhezkazgan is NL International.
------------
Closest hotel to Arinos is Estampariacções e Estampar.
------------
Closest hotel to Corinth is Kkataskeuastikē.
------------
Closest hotel to Elena is pharmacy Ceiba.
------------


In [10]:
# NOTE: Do *** change *some* of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name (here)
hotel_info = [info_box_template.format(**row) for index, row in finest_cities.iterrows()]
#locations = hotel_df[["Lat", "Lng"]]
hotel_info

['\n<dl>\n<dt>Name</dt><dd>Câmara Municipal de Bataypora</dd>\n<dt>City</dt><dd>Bataiporã</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>Zabul Province</dd>\n<dt>Country</dt><dd>AF</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>NL International</dd>\n<dt>City</dt><dd>Zhezkazgan</dd>\n<dt>Country</dt><dd>KZ</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Estampariacções e Estampar</dd>\n<dt>City</dt><dd>Arinos</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Kkataskeuastikē</dd>\n<dt>City</dt><dd>Corinth</dd>\n<dt>Country</dt><dd>GR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>pharmacy Ceiba</dd>\n<dt>City</dt><dd>Elena</dd>\n<dt>Country</dt><dd>BG</dd>\n</dl>\n']

In [11]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))